# Fine-tuning

Motivation:
* Adaptation of pre-trained Transformer models to a specific task
* Leveraging models trained on massive datasets
* Saving time and power

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

Taking a look at the content of the Europarl-ST dataset having English as a source language

In [3]:
from datasets import load_dataset

raw_datasets = load_dataset("tj-solergibert/Europarl-ST-processed-mt-en")
print(raw_datasets)

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang'],
        num_rows: 86170
    })
    train: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang'],
        num_rows: 602605
    })
    valid: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang'],
        num_rows: 81968
    })
})


The original dataset includes the translation from English into several European languages

In [4]:
raw_datasets["train"].features

{'source_text': Value(dtype='string', id=None),
 'dest_text': Value(dtype='string', id=None),
 'dest_lang': ClassLabel(names=['de', 'en', 'es', 'fr', 'it', 'nl', 'pl', 'pt', 'ro'], id=None)}

Let us take a look at the translations of the first English sentence

In [ ]:
raw_datasets["train"][:7]["source_text"]

['Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.']

In [ ]:
raw_datasets["train"][:7]["dest_text"]

['Seit 1977 wurden die meisten Finanzdienstleistungen, einschließlich Versicherungen und Verwaltung von Investmentfunds, von der Mehrwertsteuer ausgenommen.',
 'La mayoría de los servicios financieros, incluidos los seguros y la gestión de fondos de inversión, están exentos de IVA desde 1977.',
 'Depuis 1977, la plupart des services financiers, dont les assurances et la gestion des fonds de placement, ne sont pas tenus d ’ appliquer une TVA.',
 'Dal 1997 la maggior parte dei servizi finanziari, compresi i servizi assicurativi e la gestione di fondi di investimento, sono esenti da IVA.',
 'Sinds 1977 zijn de meeste financiële diensten, met inbegrip van verzekeringen en het beheer van beleggingsfondsen, vrijgesteld van btw.',
 'większość usług finansowych, w tym usług w zakresie ubezpieczeń i zarządzania funduszami inwestycyjnymi, była zwolniona z opodatkowania podatkiem VAT.',
 'Desde 1977 que a maioria dos serviços financeiros, incluindo os seguros e a gestão de fundos de investimento,

In [ ]:
raw_datasets["train"][:7]["dest_lang"]

[0, 2, 3, 4, 5, 6, 7]

Filtering Europarl-ST only for English into Spanish using a simple [lambda function](https://realpython.com/python-lambda/) with the [Dataset.filter() function](https://huggingface.co/docs/datasets/v2.9.0/en/package_reference/main_classes#datasets.Dataset.filter) and taking a small sample with [Dataset.select()](https://huggingface.co/docs/datasets/v2.9.0/en/package_reference/main_classes#datasets.Dataset.select) functions together

In [4]:
lang="es"
lang_id = raw_datasets["train"].features["dest_lang"].names.index(lang)
raw_datasets["train"] = raw_datasets["train"].filter(lambda x: x["dest_lang"] == lang_id).select(range(128))
raw_datasets["valid"] = raw_datasets["valid"].filter(lambda x: x["dest_lang"] == lang_id).select(range(32))
raw_datasets["test"] = raw_datasets["test"].filter(lambda x: x["dest_lang"] == lang_id).select(range(32))

  0%|          | 0/82 [00:00<?, ?ba/s]

  0%|          | 0/87 [00:00<?, ?ba/s]

A good practice is to take a small random sample to get a quick feel for the type of data you’re working with. In Datasets, we can create a random sample by using the [Dataset.shuffle()](https://huggingface.co/docs/datasets/v2.9.0/en/package_reference/main_classes#datasets.Dataset.shuffle):

In [7]:
raw_sample = raw_datasets["train"].shuffle(seed=13).select(range(5))
raw_sample[:5]

{'source_text': ['During this period, two problems have essentially arisen: the definition of the scope of the exemption and the impossibility of recovering VAT incurred in order to provide exempt services, giving rise to the phenomenon of hidden VAT.',
  'They must also ensure adherence at home to the principle of respect for human rights and the rule of law which President Medvedev himself proclaimed at the start of his term of office.',
  'The euro cannot simply act as a safety anchor, but must also be an engine that can drive growth. The euro area and Economic and Monetary Union must be capable of responding to these challenges.',
  'To end, I must say to you, Madam Vice-President, that you already know that you can count on the support of this Parliament.',
  'starts the negotiations with a view to concluding an agreement or partnership with Russia without that country fully complying with and respecting the agreements that it signed with the European Union on 12 August and 8 Sept

Now we load the pre-trained tokenizer for the mT5 model and apply it to a sample English-Spanish pair

In [5]:
from transformers import AutoConfig, AutoTokenizer

checkpoint = "google/mt5-small"
config = AutoConfig.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
input_ids = tokenizer(raw_datasets["train"][0]["source_text"])
print(input_ids)
labels = tokenizer(raw_datasets["train"][0]["dest_text"]).input_ids
print(labels)

{'input_ids': [259, 33644, 31649, 261, 2250, 259, 18703, 5454, 261, 259, 7035, 17789, 305, 37624, 6808, 10585, 261, 783, 2101, 35893, 270, 702, 57675, 260, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
[501, 7857, 1703, 269, 595, 259, 26075, 39015, 337, 261, 33452, 1513, 595, 259, 28023, 263, 259, 276, 283, 259, 318, 27974, 269, 7242, 337, 269, 281, 28949, 261, 1957, 272, 2121, 1401, 337, 269, 259, 34418, 259, 4400, 31649, 260, 1]


In [11]:
print(tokenizer.convert_ids_to_tokens(labels))

['▁La', '▁mayor', 'ía', '▁de', '▁los', '▁', 'servicios', '▁financier', 'os', ',', '▁inclui', 'dos', '▁los', '▁', 'seguro', 's', '▁', 'y', '▁la', '▁', 'g', 'estión', '▁de', '▁fond', 'os', '▁de', '▁in', 'versión', ',', '▁está', 'n', '▁ex', 'ent', 'os', '▁de', '▁', 'IVA', '▁', 'desde', '▁1977', '.', '</s>']


In [12]:
tokenizer.decode(labels)

'La mayoría de los servicios financieros, incluidos los seguros y la gestión de fondos de inversión, están exentos de IVA desde 1977.</s>'

We can apply the tokenizer function to any dataset taking advantage that Hugging Face Datasets are [Apache Arrow](https://arrow.apache.org) files stored on the disk, so you only keep the samples you ask for loaded in memory.

To keep the data as a dataset, we will use the Dataset.map() method. This also allows us some extra flexibility, if we need more preprocessing done than just tokenization. The map() method works by applying a function on each element of the dataset, so let us define a function that tokenizes our inputs:

In [7]:
def tokenize_function(sample):
    model_inputs = tokenizer(sample["source_text"],max_length=40,truncation=True)
    model_inputs['labels'] = tokenizer(sample["dest_text"],max_length=40,truncation=True).input_ids
    return model_inputs

In [8]:
tokenized_sample = tokenize_function(raw_datasets["train"][0])
print(tokenized_sample)

{'input_ids': [259, 33644, 31649, 261, 2250, 259, 18703, 5454, 261, 259, 7035, 17789, 305, 37624, 6808, 10585, 261, 783, 2101, 35893, 270, 702, 57675, 260, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [501, 7857, 1703, 269, 595, 259, 26075, 39015, 337, 261, 33452, 1513, 595, 259, 28023, 263, 259, 276, 283, 259, 318, 27974, 269, 7242, 337, 269, 281, 28949, 261, 1957, 272, 2121, 1401, 337, 269, 259, 34418, 259, 4400, 1]}


The way the Datasets library applies this processing is by adding new fields to the datasets, one for each key in the dictionary returned by the tokenize function:

In [9]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    test: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 32
    })
    train: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 128
    })
    valid: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 32
    })
})

The function that is responsible for putting together samples inside a batch is called a collate function. It is an argument you can pass when you build a DataLoader, the default being a function that will just convert your samples to PyTorch tensors and concatenate them. This is not possible in our case since the inputs we have are not all of the same size. We have deliberately postponed the padding, to only apply it as necessary on each batch and avoid having over-long inputs with a lot of padding. 

To do this in practice, we have to define a collate function that will apply the correct amount of padding to the items of the dataset we want to batch together. Fortunately, the Transformers library provides us with such a function via DataCollatorForSeq2Seq It takes a tokenizer when you instantiate it (to know which padding token to use, and whether the model expects padding to be on the left or on the right of the inputs) and will do everything you need:

In [10]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

To take a look at how this function works, now we have our dataset encoded, let us select only those keys that will be needed to create the batch discarding the textual features.

In [12]:
samples = [tokenized_datasets["train"][i] for i in range(2)]
print(samples)
for i in range(len(samples)):
    samples[i] = {key: value for key, value in samples[i].items() if key not in ["source_text", "dest_text", "dest_lang"]}
print(samples)

[{'source_text': 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.', 'dest_text': 'La mayoría de los servicios financieros, incluidos los seguros y la gestión de fondos de inversión, están exentos de IVA desde 1977.', 'dest_lang': 2, 'input_ids': [259, 33644, 31649, 261, 2250, 259, 18703, 5454, 261, 259, 7035, 17789, 305, 37624, 6808, 10585, 261, 783, 2101, 35893, 270, 702, 57675, 260, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [501, 7857, 1703, 269, 595, 259, 26075, 39015, 337, 261, 33452, 1513, 595, 259, 28023, 263, 259, 276, 283, 259, 318, 27974, 269, 7242, 337, 269, 281, 28949, 261, 1957, 272, 2121, 1401, 337, 269, 259, 34418, 259, 4400, 1]}, {'source_text': 'During this period, two problems have essentially arisen: the definition of the scope of the exemption and the impossibility of recovering VAT incurred in order to provide exempt services, giving

In [13]:
batch = data_collator(samples)
print(batch)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[   259,  33644,  31649,    261,   2250,    259,  18703,   5454,    261,
            259,   7035,  17789,    305,  37624,   6808,  10585,    261,    783,
           2101,  35893,    270,    702,  57675,    260,      1,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0],
        [  9155,    347,    714,   8192,    261,   2956,  20276,    783,    259,
          27155,    484,    798,  12190,    267,    287,  48906,    304,    287,
            259,  32443,    304,    287,  35893,   2746,    305,    287,    259,
         155924,   2302,    304, 109667,    347,  57675, 106543,   4196,    281,
           5411,    288,   4996,      1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1,

## Evaluation

The last thing to define for our Seq2SeqTrainer is how to compute the metrics from the predictions. We need to define a function for this, which will just use the metric we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [14]:
!pip install sacrebleu
from evaluate import load

metric = load("sacrebleu")
dest_preds = ["Esto es un ejemplo de cálculo de BLEU.", "Esto es otro."]
dest_labels = [["Este es un ejemplo de cálculo de BLEU."], ["Este es otro."]]
metric.compute(predictions=dest_preds, references=dest_labels)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


{'score': 78.75110621102682,
 'counts': [11, 9, 7, 5],
 'totals': [13, 11, 9, 7],
 'precisions': [84.61538461538461,
  81.81818181818181,
  77.77777777777777,
  71.42857142857143],
 'bp': 1.0,
 'sys_len': 13,
 'ref_len': 13}

In [15]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

## Training

The first step before we can define our Trainer is to define a TrainingArguments class that will contain all the hyperparameters the Trainer will use for training and evaluation. The only argument you have to provide is a directory where the trained model will be saved, as well as the checkpoints along the way. For all the rest, you can leave the defaults, which should work pretty well for a basic fine-tuning.

In [16]:
from transformers import Seq2SeqTrainingArguments

batch_size = 16
model_name = checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-en-to-es",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


This is because mT5 has not been pretrained on translation, so the head of the pretrained model has been discarded and a new head suitable for sequence classification has been added instead. The warnings indicate that some weights were not used (the ones corresponding to the dropped pretraining head) and that some others were randomly initialized (the ones for the new head). It concludes by encouraging you to train the model, which is exactly what we are going to do now.

Once we have our model, we can define a Trainer by passing it all the objects constructed up to now — the model, the training_args, the training and validation datasets, our data_collator, and our tokenizer:

In [17]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


To fine-tune the model on our dataset, we just have to call the train() method of our Trainer:

In [18]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: source_text, dest_text, dest_lang. If source_text, dest_text, dest_lang are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 128
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 8
  Number of trainable parameters = 300176768


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,24.888845,0.057000,2.875000


The following columns in the evaluation set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: source_text, dest_text, dest_lang. If source_text, dest_text, dest_lang are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 32
  Batch size = 16
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=8, training_loss=26.019567489624023, metrics={'train_runtime': 350.8234, 'train_samples_per_second': 0.365, 'train_steps_per_second': 0.023, 'total_flos': 5287496908800.0, 'train_loss': 26.019567489624023, 'epoch': 1.0})